In [1]:
from super_gradients.training import Trainer
from super_gradients.training import dataloaders
from super_gradients.training.dataloaders.dataloaders import coco_detection_yolo_format_train, coco_detection_yolo_format_val

The console stream is logged into C:\Users\ravit\sg_logs\console.log


[2023-08-29 08:50:30] INFO - crash_tips_setup.py - Crash tips is enabled. You can set your environment variable to CRASH_HANDLER=FALSE to disable it
[2023-08-29 08:50:31] WARNING - redirects.py - NOTE: Redirects are currently not supported in Windows or MacOs.
c:\Users\ravit\anaconda3\envs\project_ibmhackathon\lib\site-packages\_distutils_hack\__init__.py:33: UserWarning: Setuptools is replacing distutils.
  warnings.warn("Setuptools is replacing distutils.")
[2023-08-29 08:50:36] WARNING - __init__.py - Failed to import pytorch_quantization
[2023-08-29 08:50:36] WARNING - calibrator.py - Failed to import pytorch_quantization
[2023-08-29 08:50:36] WARNING - export.py - Failed to import pytorch_quantization
[2023-08-29 08:50:36] WARNING - selective_quantization_utils.py - Failed to import pytorch_quantization
[2023-08-29 08:50:36] WARNING - env_sanity_check.py - Failed to verify operating system: Deci officially supports only Linux kernels. Some features may not work as expected.


In [2]:
from IPython.display import clear_output
from super_gradients.training.losses import PPYoloELoss
from super_gradients.training.metrics import DetectionMetrics_050
from super_gradients.training.models.detection_models.pp_yolo_e import PPYoloEPostPredictionCallback

In [3]:
from super_gradients.training import models

In [4]:
CHECKPOINT_DIR = 'D://PROJECTS/ibm_hackathon/project_industry/newnew_data/newcheckpoints'
trainer = Trainer(experiment_name='yolo_nas', ckpt_root_dir=CHECKPOINT_DIR)

In [5]:

dataset_params = {
    'data_dir':'D:/PROJECTS/ibm_hackathon/project_industry/newnew_data',
    'train_images_dir':'D:/PROJECTS/ibm_hackathon/project_industry/newnew_data/data/train/images',
    'train_labels_dir':'D:/PROJECTS/ibm_hackathon/project_industry/newnew_data/data/train/labels',
    'val_images_dir':'D:/PROJECTS/ibm_hackathon/project_industry/newnew_data/data/valid/images',
    'val_labels_dir':'D:/PROJECTS/ibm_hackathon/project_industry/newnew_data/data/valid/labels',
    'test_images_dir':'D:/PROJECTS/ibm_hackathon/project_industry/newnew_data/data/test/images',
    'test_labels_dir':'D:/PROJECTS/ibm_hackathon/project_industry/newnew_data/data/test/labels',
    'classes':['Fall Detected', 'Fire', 'Person', 'Smoke', 'Vest', 'hard hat']
}

In [6]:
train_data = coco_detection_yolo_format_train(
    dataset_params={
        'data_dir': dataset_params['data_dir'],
        'images_dir': dataset_params['train_images_dir'],
        'labels_dir': dataset_params['train_labels_dir'],
        'classes': dataset_params['classes']
    },
    dataloader_params={
        'batch_size':8,
        'num_workers':0
    }
)

val_data = coco_detection_yolo_format_val(
    dataset_params={
        'data_dir': dataset_params['data_dir'],
        'images_dir': dataset_params['val_images_dir'],
        'labels_dir': dataset_params['val_labels_dir'],
        'classes': dataset_params['classes']
    },
    dataloader_params={
        'batch_size':8,
        'num_workers':0
    }
)

test_data = coco_detection_yolo_format_val(
    dataset_params={
        'data_dir': dataset_params['data_dir'],
        'images_dir': dataset_params['test_images_dir'],
        'labels_dir': dataset_params['test_labels_dir'],
        'classes': dataset_params['classes']
    },
    dataloader_params={
        'batch_size':8,
        'num_workers':0
    }
)

clear_output()

Caching annotations: 100%|██████████| 34/34 [00:00<00:00, 838.39it/s]


In [7]:

model = models.get('yolo_nas_l', 
                   num_classes=len(dataset_params['classes']), 
                   pretrained_weights="coco"
                   )
     

[2023-08-29 08:50:47] INFO - checkpoint_utils.py - License Notification: YOLO-NAS pre-trained weights are subjected to the specific license terms and conditions detailed in 
https://github.com/Deci-AI/super-gradients/blob/master/LICENSE.YOLONAS.md
By downloading the pre-trained weight files you agree to comply with these terms.


In [20]:
train_params = {
    # ENABLING SILENT MODE
    'silent_mode': False,
    "average_best_models":True,
    "warmup_mode": "linear_epoch_step",
    "warmup_initial_lr": 1e-5,
    "lr_warmup_epochs": 4,
    "initial_lr": 5e-3,
    "lr_mode": "cosine",
    "cosine_final_lr_ratio": 0.2,
    "optimizer": "Adam",
    "optimizer_params": {"weight_decay": 0.00001},
    "zero_weight_decay_on_bias_and_bn": True,
    "ema": True,
    "ema_params": {"decay": 0.9, "decay_type": "threshold"},
    # ONLY TRAINING FOR 10 EPOCHS FOR THIS EXAMPLE NOTEBOOK
    "max_epochs":60,
    "mixed_precision": True,
    "loss": PPYoloELoss(
        use_static_assigner=False,
        # NOTE: num_classes needs to be defined here
        num_classes=len(dataset_params['classes']),
        reg_max=16
    ),
    "valid_metrics_list": [
        DetectionMetrics_050(
            score_thres=0.1,
            top_k_predictions=300,
            # NOTE: num_classes needs to be defined here
            num_cls=len(dataset_params['classes']),
            normalize_targets=True,
            post_prediction_callback=PPYoloEPostPredictionCallback(
                score_threshold=0.01,
                nms_top_k=1000,
                max_predictions=300,
                nms_threshold=0.7
            )
        )
    ],
    "metric_to_watch": 'mAP@0.50'
}


In [21]:
 best_model = models.get('yolo_nas_l',
                        num_classes=len(dataset_params['classes']),
                        checkpoint_path="D://PROJECTS/ibm_hackathon/project_industry/newnew_data/checkpoints/yolo_nas/ckpt_best.pth")
# this part of code is used to load the model from the checkpoints 

[2023-08-29 11:50:44] INFO - checkpoint_utils.py - Successfully loaded model weights from D://PROJECTS/ibm_hackathon/project_industry/newnew_data/checkpoints/yolo_nas/ckpt_best.pth EMA checkpoint.


In [ ]:
img_url = 'D:/PROJECTS/ibm_hackathon/project_industry/newnew_data/data/p1.jpeg'
best_model.predict(img_url).show()   # for predicting the images

In [ ]:
import torch
device = 0 if torch.cuda.is_available() else "cpu"

input_video_path = "D:/PROJECTS/ibm_hackathon/project_industry/v3.mp4" # input location of video
output_video_path = "D:/PROJECTS/ibm_hackathon/project_industry/v9.mp4" # output storage of video 
#device=0

best_model.to(device).predict(input_video_path).save(output_video_path)   # predicting and labeling in the input video and then it gets saved in the output location